In [1]:
#Read Data From CSV file
import pandas as pd
df=pd.read_csv('NEWDATATABLE.csv', sep=',',header=None)
ratings = df.values

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def train_test_generation(ratings):
    #Cretae train and test matrix
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in xrange(ratings.shape[0]):
        temp = list(ratings[user, :])
        temp_list = []
        count = 0
        # Choose top 10 known to be in test case
        for item in temp:
            if item != 99:
                temp_list.append(item)
        temp_list = sorted(temp_list, reverse=True)  
        while count < 10:
            count = count + 1
            Index_temp = temp.index(temp_list[count])
            #Set chosen be zero in training
            train [user, Index_temp] = 0.
            #Assign chosen rating to test case
            test[user, Index_temp] = ratings[user, Index_temp]
            ratings[user, Index_temp] = 0.
            temp = list(ratings[user, :])
            temp_list = []
            for item in temp:
                if item != 99:
                    temp_list.append(item)
            temp_list = sorted(temp_list, reverse=True)  
              
    return train, test

In [4]:
#Call the function and then assign train and test
train, test = train_test_generation(ratings)
trainMAE = train.copy()
#Masked for CF and not masked for SVD
train = np.ma.masked_where(train == 99, train)
test = np.ma.masked_where(test == 99, test)

In [5]:
#Find averge of each column
train_average_column = np.zeros(100)
for indexI in range(100):
    tempsum = 0;
    count = 0; 
    for indexJ in range(24983):
        if train[indexJ][indexI] != 99:
            tempsum = tempsum + train[indexJ][indexI]
            count = count + 1
    train_average_column[indexI] = float(tempsum) / count
#Find averge of each row
train_average_row = np.zeros(24983)
for indexI in range(24983):
    tempsum = 0;
    count = 0; 
    for indexJ in range(100):
        if train[indexI][indexJ] != 99:
            tempsum = tempsum + train[indexI][indexJ]
            count = count + 1
    train_average_row[indexI] = float(tempsum) / count

In [6]:
#train_norm is used for CF item/user based here
train_norm = train.copy()
#Fill the sparse with avergae of item score
for indexI in range(24983):
    for indexJ in range(100):
        if train_norm[indexI][indexJ] == 99:
            train_norm[indexI][indexJ] = train_average_column[indexJ]

In [6]:
#Fcn to compute user or item similarity
from sklearn.metrics.pairwise import cosine_similarity

def similarity(ratings, kind='user'):
    if kind == 'user':
        sim = cosine_similarity(ratings)
        assert(sim.shape[0] == ratings.shape[0])
    elif kind == 'item':
        sim = cosine_similarity(ratings.T)
        assert(sim.shape[0] == ratings.shape[1])
    sim[np.isnan(sim)] = 0
    np.fill_diagonal(sim,0)
    return sim

In [7]:
#Compute item and user similarity
item_similarity = similarity(train, kind='item')
user_similarity = similarity(train, kind='user')
#Copy them for finding index in KNN
item_similarity_fcn = item_similarity.copy()
user_similarity_fcn = user_similarity.copy()
#Copy them for computing Top K CF based
item_similarity_CF = item_similarity.copy()
user_similarity_CF = user_similarity.copy()

In [8]:
def predict(ratings, similarity, kind ='user'):
    if kind == 'user':
        user_bias = ratings.mean(axis = 1)
        ratings = (ratings - user_bias[:, np.newaxis]).copy()
        pred = np.ma.dot(similarity, ratings) / np.array([np.abs(similarity).sum(axis=1)+1e-6]).T
        pred += user_bias[:, np.newaxis]
    elif kind == 'item':
        item_bias = ratings.mean(axis = 0)
        ratings = (ratings - item_bias[np.newaxis, :]).copy()
        pred = np.ma.dot(ratings, similarity) / np.array([np.abs(similarity).sum(axis=1)+1e-6])
        pred += item_bias[np.newaxis, :]
        
    return pred

In [9]:
user_prediction = predict(train, user_similarity, kind = 'user')
item_prediction = predict(train, item_similarity, kind = 'item')

In [222]:
#Assign user_prediction or item_prediction to parameter
all_user_predicted_ratings = user_prediction

In [281]:
# 30 items for each user in test case and choose Top-N
#Pick unkonwn index
#evaluate recall and precise
unknown_index = []
exist_row = []
#discard
notexist_row = []
for indexI in range(24983):
    temp_row_index = np.where(trainMAE[indexI] == 99)
    if len(list(temp_row_index[0])) < 30 :
            notexist_row.append(indexI)
    else:
        temp_row_index_final = list(np.random.choice(temp_row_index[0], size = 30, replace = False))
        unknown_index.append(temp_row_index_final)
        exist_row.append(indexI)

In [282]:
#Find unknown rating for each user in training
from collections import defaultdict
combine = zip(exist_row, unknown_index)
row_number_result = defaultdict(list)
for indexI in range(len(combine)):
    row_temp = list()
    for item in combine[indexI][1]:
        row_temp.append(all_user_predicted_ratings[combine[indexI][0]][item])
    row_number_result[combine[indexI][0]].append(row_temp)

In [283]:
#Find known rating for each user in test
row_test_result = defaultdict(list)
for indexI in range(len(combine)):
    a = test[combine[indexI][0]]
    a = a[a != 0]
    a = list(a)
    row_test_result[combine[indexI][0]].append(a)

In [284]:
def recall_precise(index_matrix, test_index):
    recall_result = []
    precise_result = []
    N = 15
    recall_temp = 0
    precise_temp = 0
    
    for item in exist_row:
        temp_train_list = index_matrix[item][0]
        for item_test in row_test_result[item][0]:
            temp_train_list.append(item_test)
            temp_sort_list = sorted(temp_train_list, reverse = True)
            temp_index = temp_sort_list.index(item_test)
            if temp_index < N:
                count = 1
            else:
                count = 0 
            recall_temp = count
            precise_temp = float(count) / N
            recall_result.append(recall_temp)
            precise_result.append(precise_temp)
    return recall_result, precise_result

In [285]:
#Call two fcns to achieve to find
recall_each, precise_each = recall_precise(row_number_result, row_test_result)

In [286]:
recall_AVG = float(sum(recall_each)) / len(recall_each)
precise_AVG = float(sum(precise_each)) / len(precise_each)
print 'Recall: ' + str(recall_AVG)
print 'Precise: ' + str(precise_AVG)

Recall: 0.997260664569
Precise: 0.0664840443045


In [287]:
len(row_number_result)

10337